In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
NVIDIA_API_KEY = os.environ['NVIDIA_API_KEY']

In [2]:
from libs.indoxGen.indoxGen.llms import OpenAi

openai = OpenAi(api_key=OPENAI_API_KEY, model="gpt-4o-mini")

nemotron = OpenAi(api_key=NVIDIA_API_KEY, model="nvidia/nemotron-4-340b-instruct",
                  base_url="https://integrate.api.nvidia.com/v1")

In [3]:
import pandas as pd
import random


def generate_text_data():
    adjectives = ["Quick", "Lazy", "Happy", "Sad", "Brave", "Calm", "Bright", "Dark"]
    nouns = ["Fox", "Dog", "Cat", "Rabbit", "Bear", "Tiger", "Eagle", "Wolf"]
    return f"{random.choice(adjectives)} {random.choice(nouns)}"


def generate_numerical_data():
    return random.uniform(10, 100)


data = {
    "Description": [generate_text_data() for _ in range(15)],  # Random text descriptions
    "Quantity": [random.randint(1, 50) for _ in range(15)],  # Random integer values
    "Price": [round(generate_numerical_data(), 2) for _ in range(15)],  # Random float values
    "In Stock": [random.choice([True, False]) for _ in range(15)]  # Random boolean values
}

df = pd.DataFrame(data)


In [4]:
df

,Description,Quantity,Price,In Stock
0,Calm Cat,26,76.95,False
1,Sad Bear,9,86.07,True
2,Lazy Dog,50,34.45,True
3,Calm Eagle,2,24.20,False
4,Quick Eagle,10,12.66,True
5,Brave Cat,8,76.80,False
6,Lazy Wolf,43,24.68,False
7,Bright Tiger,43,56.89,True
8,Quick Rabbit,14,63.71,True
9,Calm Rabbit,29,32.04,True


In [6]:
from libs.indoxGen.indoxGen.hybrid_synth import TextTabularSynth, initialize_gan_synth, initialize_llm_synth

In [7]:
llm_columns = ["Description", "In Stock"]

llm_setup = initialize_llm_synth(generator_llm=nemotron, judge_llm=openai, real_data=df, columns=llm_columns,example_data=df,user_instruction="please only generate columns with text, leave numerical")

In [8]:

integer_columns = ["Quantity"]
mixed_columns = ["Price"]
gan_setup = initialize_gan_synth(input_dim=100,  
                                 generator_layers=[128, 256, 512],
                                 discriminator_layers=[512, 256, 128],
                                 learning_rate=2e-4,
                                 beta_1=0.5,
                                 beta_2=0.9,
                                 batch_size=128,
                                 epochs=50,
                                 n_critic=5,
                                 mixed_columns=mixed_columns,
                                 integer_columns=integer_columns,
                                 categorical_columns=None)

ImportError: Neither `indoxGen_tensor` nor `indoxGen_torch` is installed. Please install one of these packages to proceed.